### konlpy를 사용한 형태소분리와 Word2Vec를 사용한 벡터변환

In [50]:
# # 한국어 전처리를 위한
#!pip install konlpy
# !pip install Twitter
# !pip install tqdm
#!pip install gensim
# !pip install --upgrade jupyter ipywidgets

In [51]:
import os # 운영 체제 관련 작업을 수행하기 위한 Python 표준 라이브러리 중 하나 -> 자바 환경 변수를 설정하기 위해 사용
import pandas as pd # 데이터 조작 및 분석을 위한 파이썬 라이브러리 -> 데이터를 데이터프레임으로 로드하고 조작하는 데 사용
import numpy as np # 배열 및 수치 연산을 수행하기 위한 라이브러리 -> 다차원 배열 및 수학적 함수를 다루는 데 사용
from konlpy.tag import Okt # Konlpy 라이브러리에서 제공하는 한국어 형태소 분석기 중 하나 ->  Okt를 초기화하는 데 사용
from gensim.models.word2vec import Word2Vec # Gensim 라이브러리에서 제공하는 Word2Vec 모델을 생성하기 위한 클래스 -> 단어를 고차원 벡터로 표현하여 단어 간 유사성 및 의미 관계를 파악
import re #  Python의 정규 표현식(regular expression) 모듈 -> re.sub() 함수를 사용하여 문자열에서 정규 표현식 패턴에 일치하는 부분을 다른 문자열로 대체
import html # html 엔터티 코드 제거를 위해 사용

In [52]:
# 본인 컴퓨터와 자바 설치 경로 설정
os.environ["JAVA_HOME"] = "C:\Program Files\Java\jdk-11"

In [53]:
# 파일 경로 설정
file_path = '../present_data/Data0_present.csv'  # 파일 경로를 적절히 변경하세요

# pd.read_csv와 반복문을 사용하여 파일을 나누어 읽기
chunks = pd.read_csv(file_path, encoding='utf-8', chunksize=1000)

# past_data라는 빈 데이터프레임 생성
past_data = pd.DataFrame()

for chunk in chunks:
    # 각 청크에 대한 처리를 여기에 추가
    # 예를 들어, 원하는 계산 수행 또는 데이터 전처리 등을 수행

    # past_data에 현재 청크를 추가
    past_data = pd.concat([past_data, chunk], ignore_index=True)

# past_data를 사용하여 나머지 작업 수행
print(past_data.head())  # past_data의 처음 몇 행을 출력하는 예시

   musical_id       title                                         poster_url  \
0      190722       레미제라블  http://ticketimage.interpark.com/PlayDictionar...   
1      190036         레베카  http://ticketimage.interpark.com/PlayDictionar...   
2      183052     오페라의 유령  http://ticketimage.interpark.com/PlayDictionar...   
3      190718          벤허  http://ticketimage.interpark.com/PlayDictionar...   
4      190723  레미제라블 - 부산  http://ticketimage.interpark.com/PlayDictionar...   

  genre                     date                  location  \
0  라이선스  2023/11/30 ~ 2024/03/10               블루스퀘어 신한카드홀   
1  라이선스  2023/08/19 ~ 2023/11/19               블루스퀘어 신한카드홀   
2  라이선스  2023/07/21 ~ 2023/11/19                    샤롯데씨어터   
3    창작  2023/09/02 ~ 2023/11/19  LG아트센터 서울 LG SIGNATURE 홀   
4  라이선스  2023/10/11 ~ 2023/11/19                  부산 드림씨어터   

                                              actors  age_rating  \
0  민우혁,최재림,김우형,카이,조정은,린아,임기홍,육현욱,박준면,김영주,김성식,김진욱,...         NaN   
1  류정한,민영기

In [54]:
# 모든 열에 대해 특수 문자와 HTML 엔터티 코드 모두 제거
for col in past_data.columns:
    past_data[col] = past_data[col].apply(lambda x: re.sub(r'[^\w\s]', '', html.unescape(str(x))))
# 결과 출력
print(past_data)

   musical_id       title                                         poster_url  \
0      190722       레미제라블  httpticketimageinterparkcomPlayDictionaryDATAP...   
1      190036         레베카  httpticketimageinterparkcomPlayDictionaryDATAP...   
2      183052     오페라의 유령  httpticketimageinterparkcomPlayDictionaryDATAP...   
3      190718          벤허  httpticketimageinterparkcomPlayDictionaryDATAP...   
4      190723   레미제라블  부산  httpticketimageinterparkcomPlayDictionaryDATAP...   
..        ...         ...                                                ...   
80     195267        페인터즈  httpticketimageinterparkcomPlayDictionaryDATAP...   
81     167486     장수탕 선녀님  httpticketimageinterparkcomPlayDictionaryDATAP...   
82     169908  스크루테이프의 편지  httpticketimageinterparkcomPlayDictionaryDATAP...   
83     173988     라면에 파송송  httpticketimageinterparkcomPlayDictionaryDATAP...   
84     192313  테레즈 라캥  공주  httpticketimageinterparkcomPlayDictionaryDATAP...   

    genre                date          

In [55]:
past_data.to_csv('../present_data/Data1_cleaned_present.csv', index=False, encoding='utf-8')

### 텍스트 전처리

In [56]:
# 형태소 분리를 위한 Konlpy 객체 초기화
twitter = Okt()

In [57]:
#텍스트를 전처리 하는 함수
def preprocessingText(synopsis):
    #전처리한 결과를 저장 할 리스트
    stems = []
    #text 를 단어로 나누고 품사를 붙여서 리턴
    #stem=True : 단어의 기본형으로 변환  ex) 같습니다,같았습니다 -> 같다.  
    tagged_review = twitter.pos(synopsis,stem=True)

    #tagged에서 하나를 꺼내서 word,pos에 대입
    #단어는 word, 품사는 pos에 대입
    for word,pos in tagged_review:
        #len(word) : 단어의 글자수가 2글자 이상이면
        #품사가 Noun(명사) 또는 Adjective(형용사) 일경우에만 word(단어) 를 stems에 추가
        if pos=="Noun" or pos=='Adjective':
            stems.append(word)

    #리스트인 stems를 문자열로 만들어서 ( "".join(stems) ) 리턴       
    return " ".join(stems)

In [58]:
preprocessingText(past_data['synopsis'][0])

'사랑 용기 희망 대한 서사시 기구 운명 청년 장발장 빵 죄 감옥 후 가석방 전과자 대한 세상 배척 멸시 우연 기회 딘뉴 주교 고귀하다 사랑 경험 그 새롭다 인생 시작 시장 출세 어렵다 이 자상하다 온정 그 시민 사랑 존경 날 장발장 딸 양육비 위해 거리 몸 여인 판틴 구해 법 제도 맹신 자베르 경감 다시 신세 그 엄마 죽음 세상 홀로 판틴 딸 코제트 구해 수도원 잠적 세월 어느덧다 아름답다 숙녀 성장하다 코제트 마리우스 학생 혁명가 알 그 사랑 혁명 참여 마리우스 죽음 위기 장발장 의하다 목숨 구한 그 숭고하다 인간 애 박애 정신 평생 그 자베르 경감 마음 변화'

In [59]:
# 데이터프레임의 'synopsis' 열에 NaN 값이 있을 때 빈 문자열로 대체 및 전처리
past_data['synopsis_clear'] = past_data['synopsis'].fillna('').apply(preprocessingText)

In [60]:
# Word2Vec 모델 훈련을 위한 데이터 준비
sentences = past_data['synopsis'].tolist()

In [61]:
# 데이터프레임의 'synopsis' 열에 NaN 값이 있을 때 빈 문자열로 대체
tokenized_data = past_data['synopsis'].apply(lambda x: preprocessingText(str(x))).fillna('')

In [62]:
tokenized_data

0     사랑 용기 희망 대한 서사시 기구 운명 청년 장발장 빵 죄 감옥 후 가석방 전과자 ...
1     불의 사고 아내 레베카 힘드다 나날 있다 막심 드 윈터 그 몬테카를로 여행 중 사랑...
2     거부 없다 강렬하다 림 가면 속 아름답다 러브스토리 파리 오페라 하우스 무명 무용수...
3     서기 제정 로마 박해 신음 예루살렘 명망 높다 유대 귀족 벤허 로마 장교 친구 메 ...
4     사랑 용기 희망 대한 서사시 기구 운명 청년 장발장 빵 죄 감옥 후 가석방 전과자 ...
                            ...                        
80    사랑 수 없다 명의 페인터 그 환상 미술 공연 시즌 놀랍다 라이브 드로잉 감각 어 ...
81    겁 마르다 얘 나 저기 산속 선녀 란 우리 동네 오래되다 목욕탕 있다 큰길 새롭다 ...
82    경험 많다 노회 고참 악마 이자 악마 특수 공작 의 보스 스크루 테이프 원수 밤낮 ...
83    비정규직 전전 가정 가장 다훈 보증금 백만원 월세 십오만원 명가 전단지 보고 걸음 ...
84    프랑스 시절 아버지 로부터 고모 뒤 병약하다 사촌 카미유 유년 시절 테 레즈 테 레...
Name: synopsis, Length: 85, dtype: object

In [63]:
# synopsis_clear의 마지막에 " " 추가
past_data["synopsis_clear"] = past_data['synopsis_clear'].astype(str) + " "

In [64]:
#data.split(" ") : 문자열을 공백기준으로 잘라서 리스트로 변환
past_data["synopsis_clear"].apply(lambda data : data.split(" "))

0     [사랑, 용기, 희망, 대한, 서사시, 기구, 운명, 청년, 장발장, 빵, 죄, 감...
1     [불의, 사고, 아내, 레베카, 힘드다, 나날, 있다, 막심, 드, 윈터, 그, 몬...
2     [거부, 없다, 강렬하다, 림, 가면, 속, 아름답다, 러브스토리, 파리, 오페라,...
3     [서기, 제정, 로마, 박해, 신음, 예루살렘, 명망, 높다, 유대, 귀족, 벤허,...
4     [사랑, 용기, 희망, 대한, 서사시, 기구, 운명, 청년, 장발장, 빵, 죄, 감...
                            ...                        
80    [사랑, 수, 없다, 명의, 페인터, 그, 환상, 미술, 공연, 시즌, 놀랍다, 라...
81    [겁, 마르다, 얘, 나, 저기, 산속, 선녀, 란, 우리, 동네, 오래되다, 목욕...
82    [경험, 많다, 노회, 고참, 악마, 이자, 악마, 특수, 공작, 의, 보스, 스크...
83    [비정규직, 전전, 가정, 가장, 다훈, 보증금, 백만원, 월세, 십오만원, 명가,...
84    [프랑스, 시절, 아버지, 로부터, 고모, 뒤, 병약하다, 사촌, 카미유, 유년, ...
Name: synopsis_clear, Length: 85, dtype: object

In [65]:
#df["synopsis_clear"].apply(lambda data : data.split(" ")) : synopsis_clear 컬럼을 공백을 기준으로 단어별로 분리해서 리턴
past_data["tokenized_data"] = past_data["synopsis_clear"].apply(lambda data : data.split(" "))

In [66]:
past_data

,musical_id,title,poster_url,genre,date,location,actors,age_rating,running_time,describe,synopsis,synopsis_clear,tokenized_data
0,190722,레미제라블,httpticketimageinterparkcomPlayDictionaryDATAP...,라이선스,20231130 20240310,블루스퀘어 신한카드홀,민우혁최재림김우형카이조정은린아임기홍육현욱박준면김영주김성식김진욱김수하루미나윤은오김경록...,nan,180분 인터미션 20분,Les Misérables\n뮤지컬 레미제라블\n시대를 뛰어넘은\n위대한 명작\n월...,사랑 용기 희망에 대한 대서사시\r\n기구한 운명의 청년 장발장이 빵을 훔친 죄로 ...,사랑 용기 희망 대한 서사시 기구 운명 청년 장발장 빵 죄 감옥 후 가석방 전과자 ...,"[사랑, 용기, 희망, 대한, 서사시, 기구, 운명, 청년, 장발장, 빵, 죄, 감..."
1,190036,레베카,httpticketimageinterparkcomPlayDictionaryDATAP...,라이선스,20230819 20231119,블루스퀘어 신한카드홀,류정한민영기에녹테이신영숙옥주현리사장은아김보경이지혜이지수웬디윤석원임정모윤사봉김지선이은...,8세이상,170분 인터미션 20분,더 이상 수식어가 필요 없는 가장 완벽한 뮤지컬역사적 10주년 기념공연으로 돌아온 ...,불의의 사고로 아내 레베카를 잃고 힘든 나날을 보내고 있는 막심 드 윈터그는 몬테카...,불의 사고 아내 레베카 힘드다 나날 있다 막심 드 윈터 그 몬테카를로 여행 중 사랑...,"[불의, 사고, 아내, 레베카, 힘드다, 나날, 있다, 막심, 드, 윈터, 그, 몬..."
2,183052,오페라의 유령,httpticketimageinterparkcomPlayDictionaryDATAP...,라이선스,20230721 20231119,샤롯데씨어터,조승우최재림김주택전동석손지수송은혜송원근황건하윤영석이상준이지영한보라박회림김아선,초등학생이상 관람가,150분 인터미션 20분,13년 간의 긴 기다림마침내 한국의 유령이 온다환영과도 같은 무대사라지지 않을 영원...,거부할수없는 강렬한 이끌림 가면속에 감춰진 아름다운 러브스토리 파리 오페라 하우...,거부 없다 강렬하다 림 가면 속 아름답다 러브스토리 파리 오페라 하우스 무명 무용수...,"[거부, 없다, 강렬하다, 림, 가면, 속, 아름답다, 러브스토리, 파리, 오페라,..."
3,190718,벤허,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20230902 20231119,LG아트센터 서울 LG SIGNATURE 홀,박은태신성록규현이지훈박민성서경수윤공주이정화최지혜이정열홍경수한지연류수화장혜린문은수김대...,8세이상,155분 인터미션 20분,대한민국 창작 뮤지컬의 압도적 수작의 귀환제2회 한국뮤지컬어워즈 대상 무대예술상 앙...,서기 26년 제정 로마의 박해에 신음하는 예루살렘 명망 높은 유대의 귀족 벤허는 로...,서기 제정 로마 박해 신음 예루살렘 명망 높다 유대 귀족 벤허 로마 장교 친구 메 ...,"[서기, 제정, 로마, 박해, 신음, 예루살렘, 명망, 높다, 유대, 귀족, 벤허,..."
4,190723,레미제라블 부산,httpticketimageinterparkcomPlayDictionaryDATAP...,라이선스,20231011 20231119,부산 드림씨어터,민우혁최재림김우형카이조정은린아임기홍육현욱박준면김영주김성식김진욱김수하루미나윤은오김경록...,만 7세이상,nan,시대를 뛰어넘는 마스터피스10년 만의 위대한 귀환한국 라이선스 공연 10주년 기념역...,사랑 용기 희망에 대한 대서사시\r\n기구한 운명의 청년 장발장이 빵을 훔친 죄로 ...,사랑 용기 희망 대한 서사시 기구 운명 청년 장발장 빵 죄 감옥 후 가석방 전과자 ...,"[사랑, 용기, 희망, 대한, 서사시, 기구, 운명, 청년, 장발장, 빵, 죄, 감..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,195267,페인터즈,httpticketimageinterparkcomPlayDictionaryDATAP...,퍼포먼스,20231101 20231231,명보아트홀,nan,전체관람가,75분,보고나면 사랑할 수 밖에 없는 4명의 페인터즈\n그들이 펼치는 환상의 미술 공연\n...,보고나면 사랑할 수 밖에 없는 4명의 페인터즈\n그들이 펼치는 환상의 미술 공연\n...,사랑 수 없다 명의 페인터 그 환상 미술 공연 시즌 놀랍다 라이브 드로잉 감각 어 ...,"[사랑, 수, 없다, 명의, 페인터, 그, 환상, 미술, 공연, 시즌, 놀랍다, 라..."
81,167486,장수탕 선녀님,httpticketimageinterparkcomPlayDictionaryDATAP...,어린이가족,20210916 20240303,서울숲 씨어터 2관,한송이박시연한리예최승환최혜인박시온양다은이서은최성애김인섭이용호김명훈박나연변민슬권유진한...,12개월이상,60분,가족뮤지컬 연간 판매랭킹 1위 동네 낡은 목욕탕에서 만나는 신비로운 판타지의 세계...,겁먹지 마라 얘야 나는 저기 산속에 사는 선녀란다 우리 동네에는 오래된 목욕탕이...,겁 마르다 얘 나 저기 산속 선녀 란 우리 동네 오래되다 목욕탕 있다 큰길 새롭다 ...,"[겁, 마르다, 얘, 나, 저기, 산속, 선녀, 란, 우리, 동네, 오래되다, 목욕..."
82,169908,스크루테이프의 편지,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20220101 20231230,북촌나래홀,nan,만 12세이상,80분,나니아 연대기 작가 CS 루이스의 경험 사색 문학적 상상력으로 가득찬공연 뮤지컬...,경험 많고 노회한 고참 악마이자 악마특수공작대Devil Special Agency...,경험 많다 노회 고참 악마 이자 악마 특수 공작 의 보스 스크루 테이프 원수 밤낮 ...,"[경험, 많다, 노회, 고참, 악마, 이자, 악마, 특수, 공작, 의, 보스, 스크..."
83,173988,라면에 파송송,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20220423 20240127,로즈아트홀 구노방극장,김재선김슬기김태경,만 10세이상,100분,작가 의도 인생은 맛있다 누군가는 현재의 삶이 달콤할 수도 있고 누군가는 현재...,비정규직을 전전하는 한 가정의 가장 다훈 우연히 보증금 백만원과 월세 십오만원에 나...,비정규직 전전 가정 가장 다훈 보증금 백만원 월세 십오만원 명가 전단지 보고 걸음 ...,"[비정규직, 전전, 가정, 가장, 다훈, 보증금, 백만원, 월세, 십오만원, 명가,..."


In [67]:
past_data["tokenized_data"]

0     [사랑, 용기, 희망, 대한, 서사시, 기구, 운명, 청년, 장발장, 빵, 죄, 감...
1     [불의, 사고, 아내, 레베카, 힘드다, 나날, 있다, 막심, 드, 윈터, 그, 몬...
2     [거부, 없다, 강렬하다, 림, 가면, 속, 아름답다, 러브스토리, 파리, 오페라,...
3     [서기, 제정, 로마, 박해, 신음, 예루살렘, 명망, 높다, 유대, 귀족, 벤허,...
4     [사랑, 용기, 희망, 대한, 서사시, 기구, 운명, 청년, 장발장, 빵, 죄, 감...
                            ...                        
80    [사랑, 수, 없다, 명의, 페인터, 그, 환상, 미술, 공연, 시즌, 놀랍다, 라...
81    [겁, 마르다, 얘, 나, 저기, 산속, 선녀, 란, 우리, 동네, 오래되다, 목욕...
82    [경험, 많다, 노회, 고참, 악마, 이자, 악마, 특수, 공작, 의, 보스, 스크...
83    [비정규직, 전전, 가정, 가장, 다훈, 보증금, 백만원, 월세, 십오만원, 명가,...
84    [프랑스, 시절, 아버지, 로부터, 고모, 뒤, 병약하다, 사촌, 카미유, 유년, ...
Name: tokenized_data, Length: 85, dtype: object

### 유사도 측정
- 특정 줄거리와 다른 모든 줄거리 간의 유사도를 측정

In [68]:
from sklearn.metrics.pairwise import cosine_similarity

In [69]:
# 학습용 Word2Vec 모델 훈련(벡터화)
model = Word2Vec(past_data["tokenized_data"], 
                 vector_size=100, #WordVector의 열의 수
                 window=3, # 고려할 앞뒤 폭(앞뒤 3단어)
                 min_count=2, # 사용할 단어의 최소 빈도(2회 이하 단어 무시)
                 sg=1) # 0: CBOW, 1: Skip-gram

In [70]:
model.save("vector_model.bin")

In [71]:
#Vector 형태로 변환한 단어 리스트 조회
model.wv.key_to_index

{'있다': 0,
 '': 1,
 '그': 2,
 '수': 3,
 '이': 4,
 '없다': 5,
 '사랑': 6,
 '것': 7,
 '날': 8,
 '시작': 9,
 '자신': 10,
 '위해': 11,
 '친구': 12,
 '꿈': 13,
 '사람': 14,
 '집': 15,
 '말': 16,
 '이야기': 17,
 '공연': 18,
 '아이': 19,
 '마음': 20,
 '곳': 21,
 '다시': 22,
 '돼지': 23,
 '과연': 24,
 '대한': 25,
 '늑대': 26,
 '알': 27,
 '우리': 28,
 '나': 29,
 '준비': 30,
 '두': 31,
 '인생': 32,
 '순간': 33,
 '할머니': 34,
 '속': 35,
 '안': 36,
 '마술': 37,
 '때': 38,
 '모든': 39,
 '아름답다': 40,
 '후': 41,
 '새롭다': 42,
 '같다': 43,
 '위': 44,
 '최고': 45,
 '그녀': 46,
 '하나': 47,
 '남자': 48,
 '춤': 49,
 '순신': 50,
 '눈': 51,
 '가장': 52,
 '학교': 53,
 '못': 54,
 '행복하다': 55,
 '게': 56,
 '생각': 57,
 '인하다': 58,
 '이름': 59,
 '그렇다': 60,
 '앞': 61,
 '과학': 62,
 '팀': 63,
 '달타냥': 64,
 '형제': 65,
 '기억': 66,
 '밤': 67,
 '모두': 68,
 '온': 69,
 '서로': 70,
 '중': 71,
 '벤허': 72,
 '더': 73,
 '엄마': 74,
 '세상': 75,
 '사건': 76,
 '태양': 77,
 '내': 78,
 '쇼': 79,
 '시간': 80,
 '세계': 81,
 '헌': 82,
 '성공하다': 83,
 '일상': 84,
 '많다': 85,
 '그림': 86,
 '도치': 87,
 '사실': 88,
 '노래': 89,
 '통해': 90,
 '학생': 91,
 '연수': 92,
 '전': 9

In [72]:
#친구의 인덱스(빈도수 순위 0이 가장 빈도수 높은 단어)
model.wv.key_to_index["친구"]

12

In [73]:
#친구과 비슷한 벡터를 가진 단어 조회
model.wv.most_similar("친구")

[('있다', 0.8913820385932922),
 ('그', 0.8905038237571716),
 ('이', 0.8877070546150208),
 ('자신', 0.8855355978012085),
 ('마술', 0.8685287833213806),
 ('위해', 0.8681998252868652),
 ('인생', 0.8660777807235718),
 ('것', 0.8636643290519714),
 ('사람', 0.8566874861717224),
 ('수', 0.8561896085739136)]

In [74]:
#친구의 WordVector 조회
model.wv["친구"]

array([-1.4433099e-02,  8.9267911e-03,  1.0044909e-02, -5.5734832e-03,
        9.9682594e-03, -1.6467825e-02,  2.2110990e-03,  3.9959263e-02,
       -5.8682397e-04, -1.0981619e-02, -2.1150475e-02, -2.6202248e-02,
       -1.7373674e-02,  1.9755741e-03, -1.1590512e-03, -2.1442385e-02,
        1.7276538e-03, -1.3862067e-02,  5.9882081e-03, -3.3594273e-02,
       -1.0888102e-03,  6.3439072e-03,  2.9257967e-03, -1.4221438e-03,
       -7.8405030e-03,  1.3802655e-02, -2.2144845e-02, -1.1828894e-03,
       -1.0061661e-02,  1.2377453e-02,  1.0228473e-02,  2.2085260e-03,
        4.3081366e-03, -1.1065111e-02, -1.6420348e-02,  9.5527209e-03,
       -1.0351672e-02, -2.6514292e-02, -1.3655567e-02, -2.7074635e-02,
        2.0561427e-05, -7.9686064e-03, -2.5679481e-03, -3.8910406e-03,
        6.2239316e-04, -2.3855647e-02,  4.9052481e-03, -7.5271893e-03,
        1.4815850e-03,  1.6575757e-02,  1.9025913e-02, -2.4464829e-02,
        1.4923189e-03, -5.1571531e-03, -1.6094606e-02,  5.5370689e-03,
      

In [75]:
#WordVector로 변환한 단어들 word2vec_words 대입
word2vec_words = model.wv.key_to_index.keys()
word2vec_words

dict_keys(['있다', '', '그', '수', '이', '없다', '사랑', '것', '날', '시작', '자신', '위해', '친구', '꿈', '사람', '집', '말', '이야기', '공연', '아이', '마음', '곳', '다시', '돼지', '과연', '대한', '늑대', '알', '우리', '나', '준비', '두', '인생', '순간', '할머니', '속', '안', '마술', '때', '모든', '아름답다', '후', '새롭다', '같다', '위', '최고', '그녀', '하나', '남자', '춤', '순신', '눈', '가장', '학교', '못', '행복하다', '게', '생각', '인하다', '이름', '그렇다', '앞', '과학', '팀', '달타냥', '형제', '기억', '밤', '모두', '온', '서로', '중', '벤허', '더', '엄마', '세상', '사건', '태양', '내', '쇼', '시간', '세계', '헌', '성공하다', '일상', '많다', '그림', '도치', '사실', '노래', '통해', '학생', '연수', '전', '힘', '둘', '결혼', '달', '무지개', '삼총사', '오늘', '살', '하루', '때문', '음악', '우주', '세', '명의', '돈', '용기', '연습생', '밴드', '마을', '이다', '희다', '채', '유령', '숲', '바로', '가게', '여자', '사탕', '요리', '거', '회사', '파티', '당하다', '신데렐라', '시절', '전쟁', '듯', '도로시', '아니다', '위기', '최종', '사이', '아기', '지금', '소희', '네', '청년', '파리', '인간', '손님', '앤더슨', '로리스', '목', '이상하다', '번개', '일', '커플', '소식', '딸', '악마', '방', '처영', '댄스', '좋다', '다른', '아빠', '상황', '탄생', '보이지', '똥', '소리', '크리스틴', '작다', '팀원', '무엇

In [76]:
word2vec_words

dict_keys(['있다', '', '그', '수', '이', '없다', '사랑', '것', '날', '시작', '자신', '위해', '친구', '꿈', '사람', '집', '말', '이야기', '공연', '아이', '마음', '곳', '다시', '돼지', '과연', '대한', '늑대', '알', '우리', '나', '준비', '두', '인생', '순간', '할머니', '속', '안', '마술', '때', '모든', '아름답다', '후', '새롭다', '같다', '위', '최고', '그녀', '하나', '남자', '춤', '순신', '눈', '가장', '학교', '못', '행복하다', '게', '생각', '인하다', '이름', '그렇다', '앞', '과학', '팀', '달타냥', '형제', '기억', '밤', '모두', '온', '서로', '중', '벤허', '더', '엄마', '세상', '사건', '태양', '내', '쇼', '시간', '세계', '헌', '성공하다', '일상', '많다', '그림', '도치', '사실', '노래', '통해', '학생', '연수', '전', '힘', '둘', '결혼', '달', '무지개', '삼총사', '오늘', '살', '하루', '때문', '음악', '우주', '세', '명의', '돈', '용기', '연습생', '밴드', '마을', '이다', '희다', '채', '유령', '숲', '바로', '가게', '여자', '사탕', '요리', '거', '회사', '파티', '당하다', '신데렐라', '시절', '전쟁', '듯', '도로시', '아니다', '위기', '최종', '사이', '아기', '지금', '소희', '네', '청년', '파리', '인간', '손님', '앤더슨', '로리스', '목', '이상하다', '번개', '일', '커플', '소식', '딸', '악마', '방', '처영', '댄스', '좋다', '다른', '아빠', '상황', '탄생', '보이지', '똥', '소리', '크리스틴', '작다', '팀원', '무엇

### 벡터를 문자열로 저장

In [77]:
#행의 수를 row_num에 저장
row_num = len(past_data)
row_num

85

In [78]:
string_array = []  # 문자열을 저장할 빈 리스트

for index in range(row_num):
    # 뮤지컬 번호 조회
    NUM = past_data.loc[index,"musical_id"]
    print("NUM=",NUM)
    # 뮤지컬 제목 조회
    TITLE = past_data.loc[index,"title"]
    print("TITLE=",TITLE)
    # 줄거리 조회
    LINE = past_data.loc[index,"tokenized_data"]
    print("LINE=",LINE)
    doc2vec = None
    count = 0
    for word in LINE: #줄거리에서 단어 1개씩 리턴
        print("word=",word, end="\t")
        if word in word2vec_words: #단어가 WordVector에 포함된 단어 이면
            count += 1 #개수 1증가
            if doc2vec is None: #doc2vect이 None(초기값 맨처음 실행 첫번째 단어) 이면
                doc2vec = model.wv[word] #단어의 WordVector 값 대입
            else: #첫번째 단어 아님
                doc2vec = doc2vec + model.wv[word] #단어의 WordVector 값 추가

    if  doc2vec is not None:
        # 단어 벡터를 모두 더한 벡터의 값을 단어의 개수로 나눠줌
        doc2vec = doc2vec / count
     # 문자열로 변환해서 string_array 리스트에 추가
    string_array.append(doc2vec.tostring())

# string_array에 저장된 문자열을 데이터프레임에 추가
past_data["synopsis_vector"] = string_array

NUM= 190722
TITLE= 레미제라블
LINE= ['사랑', '용기', '희망', '대한', '서사시', '기구', '운명', '청년', '장발장', '빵', '죄', '감옥', '후', '가석방', '전과자', '대한', '세상', '배척', '멸시', '우연', '기회', '딘뉴', '주교', '고귀하다', '사랑', '경험', '그', '새롭다', '인생', '시작', '시장', '출세', '어렵다', '이', '자상하다', '온정', '그', '시민', '사랑', '존경', '날', '장발장', '딸', '양육비', '위해', '거리', '몸', '여인', '판틴', '구해', '법', '제도', '맹신', '자베르', '경감', '다시', '신세', '그', '엄마', '죽음', '세상', '홀로', '판틴', '딸', '코제트', '구해', '수도원', '잠적', '세월', '어느덧다', '아름답다', '숙녀', '성장하다', '코제트', '마리우스', '학생', '혁명가', '알', '그', '사랑', '혁명', '참여', '마리우스', '죽음', '위기', '장발장', '의하다', '목숨', '구한', '그', '숭고하다', '인간', '애', '박애', '정신', '평생', '그', '자베르', '경감', '마음', '변화', '']
word= 사랑	word= 용기	word= 희망	word= 대한	word= 서사시	word= 기구	word= 운명	word= 청년	word= 장발장	word= 빵	word= 죄	word= 감옥	word= 후	word= 가석방	word= 전과자	word= 대한	word= 세상	word= 배척	word= 멸시	word= 우연	word= 기회	word= 딘뉴	word= 주교	word= 고귀하다	word= 사랑	word= 경험	word= 그	word= 새롭다	word= 인생	word= 시작	word= 시장	word= 출세	word= 어렵다	word= 이	word= 자상하다	word= 온정	word= 그	word= 

/tmp/ipykernel_11562/3422325990.py:28: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  string_array.append(doc2vec.tostring())


In [79]:
past_data

,musical_id,title,poster_url,genre,date,location,actors,age_rating,running_time,describe,synopsis,synopsis_clear,tokenized_data,synopsis_vector
0,190722,레미제라블,httpticketimageinterparkcomPlayDictionaryDATAP...,라이선스,20231130 20240310,블루스퀘어 신한카드홀,민우혁최재림김우형카이조정은린아임기홍육현욱박준면김영주김성식김진욱김수하루미나윤은오김경록...,nan,180분 인터미션 20분,Les Misérables\n뮤지컬 레미제라블\n시대를 뛰어넘은\n위대한 명작\n월...,사랑 용기 희망에 대한 대서사시\r\n기구한 운명의 청년 장발장이 빵을 훔친 죄로 ...,사랑 용기 희망 대한 서사시 기구 운명 청년 장발장 빵 죄 감옥 후 가석방 전과자 ...,"[사랑, 용기, 희망, 대한, 서사시, 기구, 운명, 청년, 장발장, 빵, 죄, 감...",b'\r\xc1\x90\xbb\x04\x8b\xd4;\x13\xfd :\xf2u\x...
1,190036,레베카,httpticketimageinterparkcomPlayDictionaryDATAP...,라이선스,20230819 20231119,블루스퀘어 신한카드홀,류정한민영기에녹테이신영숙옥주현리사장은아김보경이지혜이지수웬디윤석원임정모윤사봉김지선이은...,8세이상,170분 인터미션 20분,더 이상 수식어가 필요 없는 가장 완벽한 뮤지컬역사적 10주년 기념공연으로 돌아온 ...,불의의 사고로 아내 레베카를 잃고 힘든 나날을 보내고 있는 막심 드 윈터그는 몬테카...,불의 사고 아내 레베카 힘드다 나날 있다 막심 드 윈터 그 몬테카를로 여행 중 사랑...,"[불의, 사고, 아내, 레베카, 힘드다, 나날, 있다, 막심, 드, 윈터, 그, 몬...",b'\xd8M\xd1\xbb[\x8d\x14<\x18\x1f\xf09v\x01\x0...
2,183052,오페라의 유령,httpticketimageinterparkcomPlayDictionaryDATAP...,라이선스,20230721 20231119,샤롯데씨어터,조승우최재림김주택전동석손지수송은혜송원근황건하윤영석이상준이지영한보라박회림김아선,초등학생이상 관람가,150분 인터미션 20분,13년 간의 긴 기다림마침내 한국의 유령이 온다환영과도 같은 무대사라지지 않을 영원...,거부할수없는 강렬한 이끌림 가면속에 감춰진 아름다운 러브스토리 파리 오페라 하우...,거부 없다 강렬하다 림 가면 속 아름답다 러브스토리 파리 오페라 하우스 무명 무용수...,"[거부, 없다, 강렬하다, 림, 가면, 속, 아름답다, 러브스토리, 파리, 오페라,...","b""\x9f\xf8\x93\xbb\x0b\x98\x00<6\xe5\x8a:\xb3t..."
3,190718,벤허,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20230902 20231119,LG아트센터 서울 LG SIGNATURE 홀,박은태신성록규현이지훈박민성서경수윤공주이정화최지혜이정열홍경수한지연류수화장혜린문은수김대...,8세이상,155분 인터미션 20분,대한민국 창작 뮤지컬의 압도적 수작의 귀환제2회 한국뮤지컬어워즈 대상 무대예술상 앙...,서기 26년 제정 로마의 박해에 신음하는 예루살렘 명망 높은 유대의 귀족 벤허는 로...,서기 제정 로마 박해 신음 예루살렘 명망 높다 유대 귀족 벤허 로마 장교 친구 메 ...,"[서기, 제정, 로마, 박해, 신음, 예루살렘, 명망, 높다, 유대, 귀족, 벤허,...",b'(\xca\xab\xbb\xf5\xc3\xc3;*H\xa68\x1c\xd5\x8...
4,190723,레미제라블 부산,httpticketimageinterparkcomPlayDictionaryDATAP...,라이선스,20231011 20231119,부산 드림씨어터,민우혁최재림김우형카이조정은린아임기홍육현욱박준면김영주김성식김진욱김수하루미나윤은오김경록...,만 7세이상,nan,시대를 뛰어넘는 마스터피스10년 만의 위대한 귀환한국 라이선스 공연 10주년 기념역...,사랑 용기 희망에 대한 대서사시\r\n기구한 운명의 청년 장발장이 빵을 훔친 죄로 ...,사랑 용기 희망 대한 서사시 기구 운명 청년 장발장 빵 죄 감옥 후 가석방 전과자 ...,"[사랑, 용기, 희망, 대한, 서사시, 기구, 운명, 청년, 장발장, 빵, 죄, 감...",b'\r\xc1\x90\xbb\x04\x8b\xd4;\x13\xfd :\xf2u\x...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,195267,페인터즈,httpticketimageinterparkcomPlayDictionaryDATAP...,퍼포먼스,20231101 20231231,명보아트홀,nan,전체관람가,75분,보고나면 사랑할 수 밖에 없는 4명의 페인터즈\n그들이 펼치는 환상의 미술 공연\n...,보고나면 사랑할 수 밖에 없는 4명의 페인터즈\n그들이 펼치는 환상의 미술 공연\n...,사랑 수 없다 명의 페인터 그 환상 미술 공연 시즌 놀랍다 라이브 드로잉 감각 어 ...,"[사랑, 수, 없다, 명의, 페인터, 그, 환상, 미술, 공연, 시즌, 놀랍다, 라...",b'b1\xda\xbb\xaf\xcb\x03<\xfa\xdd\x82\xbaWw9;\...
81,167486,장수탕 선녀님,httpticketimageinterparkcomPlayDictionaryDATAP...,어린이가족,20210916 20240303,서울숲 씨어터 2관,한송이박시연한리예최승환최혜인박시온양다은이서은최성애김인섭이용호김명훈박나연변민슬권유진한...,12개월이상,60분,가족뮤지컬 연간 판매랭킹 1위 동네 낡은 목욕탕에서 만나는 신비로운 판타지의 세계...,겁먹지 마라 얘야 나는 저기 산속에 사는 선녀란다 우리 동네에는 오래된 목욕탕이...,겁 마르다 얘 나 저기 산속 선녀 란 우리 동네 오래되다 목욕탕 있다 큰길 새롭다 ...,"[겁, 마르다, 얘, 나, 저기, 산속, 선녀, 란, 우리, 동네, 오래되다, 목욕...",b'\xc3}\xd0\xbb\x99\xe4\xc8;\xfe\xe3\x95\xb9\x...
82,169908,스크루테이프의 편지,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20220101 20231230,북촌나래홀,nan,만 12세이상,80분,나니아 연대기 작가 CS 루이스의 경험 사색 문학적 상상력으로 가득찬공연 뮤지컬...,경험 많고 노회한 고참 악마이자 악마특수공작대Devil Special Agency...,경험 많다 노회 고참 악마 이자 악마 특수 공작 의 보스 스크루 테이프 원수 밤낮 ...,"[경험, 많다, 노회, 고참, 악마, 이자, 악마, 특수, 공작, 의, 보스, 스크...",b'\xf1\xc2\xab\xbb\xc2\xd40<\x88\xbc\xd5:\x98r...
83,173988,라면에 파송송,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20220423 20240127,로즈아트홀 구노방극장,김재선김슬기김태경,만 10세이상,100분,작가 의도 인생은 맛있다 누군가는 현재의 삶이 달콤할 수도 있고 누군가는 현재...,비정규직을 전전하는 한 가정의 가장 다훈 우연히 보증금 백만원과 월세 십오만원에 나...,비정규직 전전 가정 가장 다훈 보증금 백만원 월세 십오만원 명가 전단지 보고 걸음 ...,"[비정규직, 전전, 가정, 가장, 다훈, 보증금, 백만원, 월세, 십오만원, 명가,...","b""\x14d\x0b\xbc7\xeb\x1b<\xb4(8:SP\x05;)\xea\x..."


#### date를 start_date와 end_date로 분리

In [80]:
# 날짜 형식에 맞게 잘라서 "YYYY-MM-DD" 또는 "YYYY-MM" 형식의 문자열로 만듭니다.
past_data['start_date'] = past_data['date'].str[:4] + '-' + past_data['date'].str[4:6] + '-' + past_data['date'].str[6:8]
past_data['end_date'] = past_data['date'].str[:4] + '-' + past_data['date'].str[4:6]

In [81]:
# pd.to_datetime 함수 호출 시 errors='coerce' 매개변수를 사용하여 오류가 발생한 경우 NaT(Not a Time)으로 처리
past_data['start_date'] = pd.to_datetime(past_data['start_date'], format="%Y-%m-%d", errors='coerce')
past_data['end_date'] = pd.to_datetime(past_data['end_date'], format="%Y-%m", errors='coerce')

In [82]:
past_data

,musical_id,title,poster_url,genre,date,location,actors,age_rating,running_time,describe,synopsis,synopsis_clear,tokenized_data,synopsis_vector,start_date,end_date
0,190722,레미제라블,httpticketimageinterparkcomPlayDictionaryDATAP...,라이선스,20231130 20240310,블루스퀘어 신한카드홀,민우혁최재림김우형카이조정은린아임기홍육현욱박준면김영주김성식김진욱김수하루미나윤은오김경록...,nan,180분 인터미션 20분,Les Misérables\n뮤지컬 레미제라블\n시대를 뛰어넘은\n위대한 명작\n월...,사랑 용기 희망에 대한 대서사시\r\n기구한 운명의 청년 장발장이 빵을 훔친 죄로 ...,사랑 용기 희망 대한 서사시 기구 운명 청년 장발장 빵 죄 감옥 후 가석방 전과자 ...,"[사랑, 용기, 희망, 대한, 서사시, 기구, 운명, 청년, 장발장, 빵, 죄, 감...",b'\r\xc1\x90\xbb\x04\x8b\xd4;\x13\xfd :\xf2u\x...,2023-11-30,2023-11-01
1,190036,레베카,httpticketimageinterparkcomPlayDictionaryDATAP...,라이선스,20230819 20231119,블루스퀘어 신한카드홀,류정한민영기에녹테이신영숙옥주현리사장은아김보경이지혜이지수웬디윤석원임정모윤사봉김지선이은...,8세이상,170분 인터미션 20분,더 이상 수식어가 필요 없는 가장 완벽한 뮤지컬역사적 10주년 기념공연으로 돌아온 ...,불의의 사고로 아내 레베카를 잃고 힘든 나날을 보내고 있는 막심 드 윈터그는 몬테카...,불의 사고 아내 레베카 힘드다 나날 있다 막심 드 윈터 그 몬테카를로 여행 중 사랑...,"[불의, 사고, 아내, 레베카, 힘드다, 나날, 있다, 막심, 드, 윈터, 그, 몬...",b'\xd8M\xd1\xbb[\x8d\x14<\x18\x1f\xf09v\x01\x0...,2023-08-19,2023-08-01
2,183052,오페라의 유령,httpticketimageinterparkcomPlayDictionaryDATAP...,라이선스,20230721 20231119,샤롯데씨어터,조승우최재림김주택전동석손지수송은혜송원근황건하윤영석이상준이지영한보라박회림김아선,초등학생이상 관람가,150분 인터미션 20분,13년 간의 긴 기다림마침내 한국의 유령이 온다환영과도 같은 무대사라지지 않을 영원...,거부할수없는 강렬한 이끌림 가면속에 감춰진 아름다운 러브스토리 파리 오페라 하우...,거부 없다 강렬하다 림 가면 속 아름답다 러브스토리 파리 오페라 하우스 무명 무용수...,"[거부, 없다, 강렬하다, 림, 가면, 속, 아름답다, 러브스토리, 파리, 오페라,...","b""\x9f\xf8\x93\xbb\x0b\x98\x00<6\xe5\x8a:\xb3t...",2023-07-21,2023-07-01
3,190718,벤허,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20230902 20231119,LG아트센터 서울 LG SIGNATURE 홀,박은태신성록규현이지훈박민성서경수윤공주이정화최지혜이정열홍경수한지연류수화장혜린문은수김대...,8세이상,155분 인터미션 20분,대한민국 창작 뮤지컬의 압도적 수작의 귀환제2회 한국뮤지컬어워즈 대상 무대예술상 앙...,서기 26년 제정 로마의 박해에 신음하는 예루살렘 명망 높은 유대의 귀족 벤허는 로...,서기 제정 로마 박해 신음 예루살렘 명망 높다 유대 귀족 벤허 로마 장교 친구 메 ...,"[서기, 제정, 로마, 박해, 신음, 예루살렘, 명망, 높다, 유대, 귀족, 벤허,...",b'(\xca\xab\xbb\xf5\xc3\xc3;*H\xa68\x1c\xd5\x8...,2023-09-02,2023-09-01
4,190723,레미제라블 부산,httpticketimageinterparkcomPlayDictionaryDATAP...,라이선스,20231011 20231119,부산 드림씨어터,민우혁최재림김우형카이조정은린아임기홍육현욱박준면김영주김성식김진욱김수하루미나윤은오김경록...,만 7세이상,nan,시대를 뛰어넘는 마스터피스10년 만의 위대한 귀환한국 라이선스 공연 10주년 기념역...,사랑 용기 희망에 대한 대서사시\r\n기구한 운명의 청년 장발장이 빵을 훔친 죄로 ...,사랑 용기 희망 대한 서사시 기구 운명 청년 장발장 빵 죄 감옥 후 가석방 전과자 ...,"[사랑, 용기, 희망, 대한, 서사시, 기구, 운명, 청년, 장발장, 빵, 죄, 감...",b'\r\xc1\x90\xbb\x04\x8b\xd4;\x13\xfd :\xf2u\x...,2023-10-11,2023-10-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,195267,페인터즈,httpticketimageinterparkcomPlayDictionaryDATAP...,퍼포먼스,20231101 20231231,명보아트홀,nan,전체관람가,75분,보고나면 사랑할 수 밖에 없는 4명의 페인터즈\n그들이 펼치는 환상의 미술 공연\n...,보고나면 사랑할 수 밖에 없는 4명의 페인터즈\n그들이 펼치는 환상의 미술 공연\n...,사랑 수 없다 명의 페인터 그 환상 미술 공연 시즌 놀랍다 라이브 드로잉 감각 어 ...,"[사랑, 수, 없다, 명의, 페인터, 그, 환상, 미술, 공연, 시즌, 놀랍다, 라...",b'b1\xda\xbb\xaf\xcb\x03<\xfa\xdd\x82\xbaWw9;\...,2023-11-01,2023-11-01
81,167486,장수탕 선녀님,httpticketimageinterparkcomPlayDictionaryDATAP...,어린이가족,20210916 20240303,서울숲 씨어터 2관,한송이박시연한리예최승환최혜인박시온양다은이서은최성애김인섭이용호김명훈박나연변민슬권유진한...,12개월이상,60분,가족뮤지컬 연간 판매랭킹 1위 동네 낡은 목욕탕에서 만나는 신비로운 판타지의 세계...,겁먹지 마라 얘야 나는 저기 산속에 사는 선녀란다 우리 동네에는 오래된 목욕탕이...,겁 마르다 얘 나 저기 산속 선녀 란 우리 동네 오래되다 목욕탕 있다 큰길 새롭다 ...,"[겁, 마르다, 얘, 나, 저기, 산속, 선녀, 란, 우리, 동네, 오래되다, 목욕...",b'\xc3}\xd0\xbb\x99\xe4\xc8;\xfe\xe3\x95\xb9\x...,2021-09-16,2021-09-01
82,169908,스크루테이프의 편지,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20220101 20231230,북촌나래홀,nan,만 12세이상,80분,나니아 연대기 작가 CS 루이스의 경험 사색 문학적 상상력으로 가득찬공연 뮤지컬...,경험 많고 노회한 고참 악마이자 악마특수공작대Devil Special Agency...,경험 많다 노회 고참 악마 이자 악마 특수 공작 의 보스 스크루 테이프 원수 밤낮 ...,"[경험, 많다, 노회, 고참, 악마, 이자, 악마, 특수, 공작, 의, 보스, 스크...",b'\xf1\xc2\xab\xbb\xc2\xd40<\x88\xbc\xd5:\x98r...,2022-01-01,2022-01-01
83,173988,라면에 파송송,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20220423 20240127,로즈아트홀 구노방극장,김재선김슬기김태경,만 10세이상,100분,작가 의도 인생은 맛있다 누군가는 현재의 삶이 달콤할 수도 있고 누군가는 현재...,비정규직을 전전하는 한 가정의 가장 다훈 우연히 보증금 백만원과 월세 십오만원에 나...,비정규직 전전 가정 가장 다훈 보증금 백만원 월세 십오만원 명가 전단지 보고 걸음 ...,"[비정규직, 전전, 가정, 가장, 다훈, 보증금, 백만원, 월세, 십오만원, 명가,...","b""\x14d\x0b\xbc7\xeb\x1b<\xb4(8:

In [83]:
# 'date'와 'location' 사이로 'start_date'와 'end_date' 위치 변경
past_data = past_data[['musical_id', 'title', 'poster_url', 'genre', 'date', 'start_date', 'end_date', 'location',
                             'actors', 'age_rating', 'running_time', 'describe', 'synopsis', 'synopsis_clear',
                             'tokenized_data', 'synopsis_vector']]
past_data

,musical_id,title,poster_url,genre,date,start_date,end_date,location,actors,age_rating,running_time,describe,synopsis,synopsis_clear,tokenized_data,synopsis_vector
0,190722,레미제라블,httpticketimageinterparkcomPlayDictionaryDATAP...,라이선스,20231130 20240310,2023-11-30,2023-11-01,블루스퀘어 신한카드홀,민우혁최재림김우형카이조정은린아임기홍육현욱박준면김영주김성식김진욱김수하루미나윤은오김경록...,nan,180분 인터미션 20분,Les Misérables\n뮤지컬 레미제라블\n시대를 뛰어넘은\n위대한 명작\n월...,사랑 용기 희망에 대한 대서사시\r\n기구한 운명의 청년 장발장이 빵을 훔친 죄로 ...,사랑 용기 희망 대한 서사시 기구 운명 청년 장발장 빵 죄 감옥 후 가석방 전과자 ...,"[사랑, 용기, 희망, 대한, 서사시, 기구, 운명, 청년, 장발장, 빵, 죄, 감...",b'\r\xc1\x90\xbb\x04\x8b\xd4;\x13\xfd :\xf2u\x...
1,190036,레베카,httpticketimageinterparkcomPlayDictionaryDATAP...,라이선스,20230819 20231119,2023-08-19,2023-08-01,블루스퀘어 신한카드홀,류정한민영기에녹테이신영숙옥주현리사장은아김보경이지혜이지수웬디윤석원임정모윤사봉김지선이은...,8세이상,170분 인터미션 20분,더 이상 수식어가 필요 없는 가장 완벽한 뮤지컬역사적 10주년 기념공연으로 돌아온 ...,불의의 사고로 아내 레베카를 잃고 힘든 나날을 보내고 있는 막심 드 윈터그는 몬테카...,불의 사고 아내 레베카 힘드다 나날 있다 막심 드 윈터 그 몬테카를로 여행 중 사랑...,"[불의, 사고, 아내, 레베카, 힘드다, 나날, 있다, 막심, 드, 윈터, 그, 몬...",b'\xd8M\xd1\xbb[\x8d\x14<\x18\x1f\xf09v\x01\x0...
2,183052,오페라의 유령,httpticketimageinterparkcomPlayDictionaryDATAP...,라이선스,20230721 20231119,2023-07-21,2023-07-01,샤롯데씨어터,조승우최재림김주택전동석손지수송은혜송원근황건하윤영석이상준이지영한보라박회림김아선,초등학생이상 관람가,150분 인터미션 20분,13년 간의 긴 기다림마침내 한국의 유령이 온다환영과도 같은 무대사라지지 않을 영원...,거부할수없는 강렬한 이끌림 가면속에 감춰진 아름다운 러브스토리 파리 오페라 하우...,거부 없다 강렬하다 림 가면 속 아름답다 러브스토리 파리 오페라 하우스 무명 무용수...,"[거부, 없다, 강렬하다, 림, 가면, 속, 아름답다, 러브스토리, 파리, 오페라,...","b""\x9f\xf8\x93\xbb\x0b\x98\x00<6\xe5\x8a:\xb3t..."
3,190718,벤허,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20230902 20231119,2023-09-02,2023-09-01,LG아트센터 서울 LG SIGNATURE 홀,박은태신성록규현이지훈박민성서경수윤공주이정화최지혜이정열홍경수한지연류수화장혜린문은수김대...,8세이상,155분 인터미션 20분,대한민국 창작 뮤지컬의 압도적 수작의 귀환제2회 한국뮤지컬어워즈 대상 무대예술상 앙...,서기 26년 제정 로마의 박해에 신음하는 예루살렘 명망 높은 유대의 귀족 벤허는 로...,서기 제정 로마 박해 신음 예루살렘 명망 높다 유대 귀족 벤허 로마 장교 친구 메 ...,"[서기, 제정, 로마, 박해, 신음, 예루살렘, 명망, 높다, 유대, 귀족, 벤허,...",b'(\xca\xab\xbb\xf5\xc3\xc3;*H\xa68\x1c\xd5\x8...
4,190723,레미제라블 부산,httpticketimageinterparkcomPlayDictionaryDATAP...,라이선스,20231011 20231119,2023-10-11,2023-10-01,부산 드림씨어터,민우혁최재림김우형카이조정은린아임기홍육현욱박준면김영주김성식김진욱김수하루미나윤은오김경록...,만 7세이상,nan,시대를 뛰어넘는 마스터피스10년 만의 위대한 귀환한국 라이선스 공연 10주년 기념역...,사랑 용기 희망에 대한 대서사시\r\n기구한 운명의 청년 장발장이 빵을 훔친 죄로 ...,사랑 용기 희망 대한 서사시 기구 운명 청년 장발장 빵 죄 감옥 후 가석방 전과자 ...,"[사랑, 용기, 희망, 대한, 서사시, 기구, 운명, 청년, 장발장, 빵, 죄, 감...",b'\r\xc1\x90\xbb\x04\x8b\xd4;\x13\xfd :\xf2u\x...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,195267,페인터즈,httpticketimageinterparkcomPlayDictionaryDATAP...,퍼포먼스,20231101 20231231,2023-11-01,2023-11-01,명보아트홀,nan,전체관람가,75분,보고나면 사랑할 수 밖에 없는 4명의 페인터즈\n그들이 펼치는 환상의 미술 공연\n...,보고나면 사랑할 수 밖에 없는 4명의 페인터즈\n그들이 펼치는 환상의 미술 공연\n...,사랑 수 없다 명의 페인터 그 환상 미술 공연 시즌 놀랍다 라이브 드로잉 감각 어 ...,"[사랑, 수, 없다, 명의, 페인터, 그, 환상, 미술, 공연, 시즌, 놀랍다, 라...",b'b1\xda\xbb\xaf\xcb\x03<\xfa\xdd\x82\xbaWw9;\...
81,167486,장수탕 선녀님,httpticketimageinterparkcomPlayDictionaryDATAP...,어린이가족,20210916 20240303,2021-09-16,2021-09-01,서울숲 씨어터 2관,한송이박시연한리예최승환최혜인박시온양다은이서은최성애김인섭이용호김명훈박나연변민슬권유진한...,12개월이상,60분,가족뮤지컬 연간 판매랭킹 1위 동네 낡은 목욕탕에서 만나는 신비로운 판타지의 세계...,겁먹지 마라 얘야 나는 저기 산속에 사는 선녀란다 우리 동네에는 오래된 목욕탕이...,겁 마르다 얘 나 저기 산속 선녀 란 우리 동네 오래되다 목욕탕 있다 큰길 새롭다 ...,"[겁, 마르다, 얘, 나, 저기, 산속, 선녀, 란, 우리, 동네, 오래되다, 목욕...",b'\xc3}\xd0\xbb\x99\xe4\xc8;\xfe\xe3\x95\xb9\x...
82,169908,스크루테이프의 편지,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20220101 20231230,2022-01-01,2022-01-01,북촌나래홀,nan,만 12세이상,80분,나니아 연대기 작가 CS 루이스의 경험 사색 문학적 상상력으로 가득찬공연 뮤지컬...,경험 많고 노회한 고참 악마이자 악마특수공작대Devil Special Agency...,경험 많다 노회 고참 악마 이자 악마 특수 공작 의 보스 스크루 테이프 원수 밤낮 ...,"[경험, 많다, 노회, 고참, 악마, 이자, 악마, 특수, 공작, 의, 보스, 스크...",b'\xf1\xc2\xab\xbb\xc2\xd40<\x88\xbc\xd5:\x98r...
83,173988,라면에 파송송,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20220423 20240127,2022-04-23,2022-04-01,로즈아트홀 구노방극장,김재선김슬기김태경,만 10세이상,100분,작가 의도 인생은 맛있다 누군가는 현재의 삶이 달콤할 수도 있고 누군가는 현재...,비정규직을 전전하는 한 가정의 가장 다훈 우연히 보증금 백만원과 월세 십오만원에 나...,비정규직 전전 가정 가장 다훈 보증금 백만원 월세 십오만원 명가 전단지 보고 걸음 ...,"[비정규직, 전전, 가정, 가장, 다훈, 보증금, 백만원, 월세, 십오만원, 명가,...","b""\x14d\x0

In [84]:
# 벡터변환 csv저장
past_data.to_csv('../present_data/Data2_present_vector.csv', index=False, encoding='utf-8')